In [17]:
import cv2
import pytesseract
import spacy
from transformers import MarianMTModel, MarianTokenizer
from langdetect import detect

In [18]:
# Load spaCy for sentence segmentation
nlp = spacy.load("en_core_web_sm")

In [19]:
# Load translation models
model_en_ar = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-ar")
tokenizer_en_ar = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ar")

model_ar_en = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
tokenizer_ar_en = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [20]:
def translate_text(text, src_lang):
    if src_lang == "en":
        tokenizer, model = tokenizer_en_ar, model_en_ar
    elif src_lang == "ar":
        tokenizer, model = tokenizer_ar_en, model_ar_en
    else:
        return ""

    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    translated = model.generate(**tokens)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

In [21]:
def detect_lang(text):
    try:
        return detect(text)
    except:
        return "unknown"

In [22]:
def ocr_and_translate(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray, lang="eng+ara")

    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]

    results = []
    for sent in sentences:
        lang = detect_lang(sent)
        if lang in ["en", "ar"]:
            translated = translate_text(sent, lang)
            results.append((sent, translated))
    return results

In [23]:
# Real-time video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    translations = ocr_and_translate(frame)

    y = 30
    for original, translated in translations:
        text_display = f"{original} => {translated}"
        cv2.putText(frame, text_display, (10, y), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 255, 0), 1, cv2.LINE_AA)
        y += 30

    cv2.imshow("English toArabic OCR Translation", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [1]:
import cv2
import pytesseract
from pytesseract import Output
from transformers import MarianMTModel, MarianTokenizer
import torch

# -----------------------------
# Load Machine Translation Model
# -----------------------------
class TranslationModel:
    def __init__(self):
        model_name = "Helsinki-NLP/opus-mt-en-ar"  # You can change this
        self.tokenizer = MarianTokenizer.from_pretrained(model_name)
        self.model = MarianMTModel.from_pretrained(model_name)

    def translate(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            translated = self.model.generate(**inputs)
        return self.tokenizer.decode(translated[0], skip_special_tokens=True)

# Create model instance
mtmodel = TranslationModel()

# -----------------------------
# OCR + Translation Function
# -----------------------------
def process_frame_with_ocr(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Tesseract config for better detection
    custom_config = r'--oem 3 --psm 6'
    data = pytesseract.image_to_data(gray, lang='eng+ara', config=custom_config, output_type=Output.DICT)

    n_boxes = len(data['text'])
    for i in range(n_boxes):
        text = data['text'][i].strip()
        if text:
            (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])

            # Translate using your model
            translated = mtmodel.translate(text)

            # Draw box and put text
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            display_text = f"{text} => {translated}"
            cv2.putText(frame, display_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (255, 0, 0), 1, cv2.LINE_AA)

    return frame

# -----------------------------
# Webcam Loop
# -----------------------------
def main():
    cap = cv2.VideoCapture(0)
    print("Press SPACE to translate frame. Press Q to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("Live Camera", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord(' '):  # Press Space to process
            processed = process_frame_with_ocr(frame.copy())
            cv2.imshow("Translated Frame", processed)
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# -----------------------------
# Run
# -----------------------------
if __name__ == "__main__":
    main()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Press SPACE to translate frame. Press Q to quit.


In [28]:
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Translate function
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Example usage
print(translate("Hello, how are you?"))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


مرحباً، كيف حالك؟
